In [1]:
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "5" # GPU - selection
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [2]:
import os
import glob
from PIL import Image
import numpy as np
import torch
from torch.utils.data import Dataset
from torchvision import transforms as T

class UrinKitDataset(Dataset):
    def __init__(self, root_dir, image_size=32):
        self.root_dir = root_dir
        self.image_size = image_size

        self.transform = T.Compose([
            T.Resize((image_size, image_size)),
            T.ToTensor()
        ])

        self.classes = sorted(os.listdir(root_dir))
        self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)}

        # 이미지-레이블 경로 추출 + 유효 키트만 추려서 저장
        self.valid_kits = []  # (img_path, label_path, class_idx, kit_index)
        for cls in self.classes:
            cls_path = os.path.join(root_dir, cls)
            for sub in os.listdir(cls_path):
                sub_path = os.path.join(cls_path, sub)
                images = glob.glob(os.path.join(sub_path, "*.jpg"))
                for img_path in images:
                    label_path = img_path.replace(".jpg", ".txt")
                    if not os.path.exists(label_path):
                        continue

                    image = Image.open(img_path).convert("RGB")
                    img_w, img_h = image.size
                    with open(label_path, "r") as f:
                        lines = list(map(str.strip, f.readlines()))
                    labels = []
                    for line in lines:
                        cls_id, x, y, w, h = map(float, line.split())
                        if cls_id != 0:
                            continue
                        x1 = (x - w / 2) * img_w
                        y1 = (y - h / 2) * img_h
                        x2 = (x + w / 2) * img_w
                        y2 = (y + h / 2) * img_h
                        labels.append([x1, y1, x2, y2, int(cls_id)])

                    labels = np.array(labels)

                    # sorted_labels = labels[np.lexsort((labels[:,1], labels[:,0]))]
                    # kit_boxes = sorted_labels[:5]
                    # sensor_boxes = sorted_labels[5:]
                    # kit_boxes = kit_boxes[np.lexsort((kit_boxes[:,0], kit_boxes[:,1]))]

                    for kit_idx, kit in enumerate(labels):
                        x1, y1, x2, y2, _ = kit
                        self.valid_kits.append((img_path, label_path, self.class_to_idx[cls], kit_idx))

    def __len__(self):
        return len(self.valid_kits)

    def __getitem__(self, idx):
        img_path, label_path, class_idx, kit_idx = self.valid_kits[idx]

        image = Image.open(img_path).convert("RGB")
        np_image = np.array(image)
        img_w, img_h = image.size

        with open(label_path, "r") as f:
            label_lines = list(map(str.strip, f.readlines()))

        labels = []
        for line in label_lines:
            cls_id, x, y, w, h = map(float, line.split())
            if cls_id != 0:
                continue
            x1 = (x - w / 2) * img_w
            y1 = (y - h / 2) * img_h
            x2 = (x + w / 2) * img_w
            y2 = (y + h / 2) * img_h
            labels.append([x1, y1, x2, y2, int(cls_id)])

        labels = np.array(labels)
        # sorted_labels = labels[np.lexsort((labels[:,1], labels[:,0]))]
        # kit_boxes = sorted_labels[:5]
        # sensor_boxes = sorted_labels[5:]

        # kit_boxes = kit_boxes[np.lexsort((kit_boxes[:,0], kit_boxes[:,1]))]
        current_kit = labels[kit_idx]

        x1, y1, x2, y2, _ = current_kit
        # mask = (sensor_boxes[:, 0] > x1) & (sensor_boxes[:, 0] < x2) & \
        #        (sensor_boxes[:, 1] > y1) & (sensor_boxes[:, 1] < y2)
        # group = sensor_boxes[mask]
        # group = group[np.lexsort((group[:,1], group[:,0]))]

        # patches = []
        # for patch in group:
        #     px1, py1, px2, py2, _ = patch
        cropped = np_image[round(y1):round(y2), round(x1):round(x2), ::-1]
        cropped_pil = Image.fromarray(cropped)
        transformed = self.transform(cropped_pil)
        # patches.append(transformed)

        # X = torch.stack(patches)  # (11, 3, H, W)
        Y = torch.tensor(class_idx)

        return transformed, Y


In [3]:
from torch.utils.data import DataLoader

dataset = UrinKitDataset("/home/jupyter-heypaprika/input/urin_by_class", image_size=224)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)


In [4]:
for X, Y in dataloader:
    print(X.shape)  # (4, 11, 3, 32, 32)
    print(Y.shape)  # (4,)
    print(Y)
    break


torch.Size([4, 3, 224, 224])
torch.Size([4])
tensor([15, 22,  9,  6])


In [5]:
import timm
import torch.nn as nn  # Neural network module

# 1. 사전학습된 FastViT 모델 로드
model = timm.create_model('fastvit_t8', pretrained=True)
# 2. 자신의 데이터셋 클래스 수에 맞게 마지막 레이어 변경
num_classes = 25  # 예: 10개 클래스 분류
model.reset_classifier(num_classes)

# Check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
model.to(device)


Using device: cuda


FastVit(
  (stem): Sequential(
    (0): MobileOneBlock(
      (se): Identity()
      (conv_kxk): ModuleList(
        (0): ConvNormAct(
          (conv): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (bn): BatchNormAct2d(
            48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): Identity()
          )
        )
      )
      (conv_scale): ConvNormAct(
        (conv): Conv2d(3, 48, kernel_size=(1, 1), stride=(2, 2), bias=False)
        (bn): BatchNormAct2d(
          48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): Identity()
        )
      )
      (act): GELU(approximate='none')
    )
    (1): MobileOneBlock(
      (se): Identity()
      (conv_kxk): ModuleList(
        (0): ConvNormAct(
          (conv): Conv2d(48, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=48, bias=False)
          (bn): Batch

In [6]:
import torch
import torch.nn as nn

class ImageSetViT(nn.Module):
    def __init__(self, patch_dim=3*32*32, embed_dim=256, num_patches=11, num_heads=4, num_classes=25, depth=4):
        super().__init__()
        self.num_patches = num_patches
        self.patch_dim = patch_dim

        # Linear embedding: 각 이미지(=패치)를 벡터로
        self.embedding = nn.Linear(patch_dim, embed_dim)

        # Learnable positional encoding
        self.pos_embedding = nn.Parameter(torch.randn(1, num_patches, embed_dim))

        # Transformer encoder
        encoder_layer = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads, batch_first=True)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=depth)

        # Classification head
        self.head = nn.Sequential(
            nn.LayerNorm(embed_dim),
            nn.Linear(embed_dim, num_classes)
        )

    def forward(self, x):
        # x: (B, 11, 3, 32, 32)
        B, N, C, H, W = x.shape
        x = x.view(B, N, -1)  # (B, 11, 3*32*32)
        x = self.embedding(x)  # (B, 11, embed_dim)
        x = x + self.pos_embedding  # positional encoding

        x = self.transformer(x)  # (B, 11, embed_dim)
        x = x.mean(dim=1)  # aggregate over patches

        out = self.head(x)  # (B, num_classes)
        return out


In [7]:
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm

# 모델 예시 (사용자 모델 정의가 이미 완료되어 있다고 가정)
# from model import ImageSetViT  
# from dataset import UrinKitDataset
model_save_dir = './files'

def train_model(model, train_loader, val_loader, device, num_epochs=10, lr=1e-4):
    model = model.to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    optimizer = optim.AdamW(model.parameters(), lr=lr)

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        correct = 0
        total = 0

        for X, Y in tqdm(train_loader, desc=f"[Train Epoch {epoch+1}]"):
            X, Y = X.to(device), Y.to(device)

            # (B, 11, 3, 32, 32) → (B, 11*3*32*32)
            # B, P, C, H, W = X.shape
            # X = X.view(B, -1)  # Flatten 11 patches

            outputs = model(X)  # Shape: (B, num_classes)
            loss = criterion(outputs, Y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == Y).sum().item()
            total += Y.size(0)

        acc = correct / total * 100
        print(f"Epoch {epoch+1}: Train Loss: {total_loss:.4f} | Train Acc: {acc:.2f}%")

        # Validation
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for X, Y in val_loader:
                X, Y = X.to(device), Y.to(device)
                # B, P, C, H, W = X.shape
                # X = X.view(B, -1)
                outputs = model(X)
                _, predicted = torch.max(outputs, 1)
                correct += (predicted == Y).sum().item()
                total += Y.size(0)

        val_acc = correct / total * 100
        print(f"Epoch {epoch+1}: Validation Accuracy: {val_acc:.2f}%\n")

        model_save_path = os.path.join(model_save_dir, f"vit_model_weight_+{epoch}epoch_{val_acc:.2f}.pth")
        torch.save(model.state_dict(), model_save_path)
        print(f'Model saved to {model_save_path}.')
    return model


In [8]:
from torch.utils.data import random_split

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
dataset = UrinKitDataset("/home/jupyter-heypaprika/input/urin_by_class", image_size=224)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_set, val_set = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_set, batch_size=12, shuffle=True)
val_loader = DataLoader(val_set, batch_size=12)

# model = ImageSetViT()  
# model.load_state_dict(torch.load('./files/vit_custom_model_weight_eth_100_2.pth', map_location=device))

trained_model = train_model(model, train_loader, val_loader, device, num_epochs=50, lr=1e-4)


cuda


[Train Epoch 1]: 100%|██████████| 399/399 [10:31<00:00,  1.58s/it]


Epoch 1: Train Loss: 908.0582 | Train Acc: 28.93%
Epoch 1: Validation Accuracy: 54.76%

Model saved to ./files/vit_model_weight_+0epoch_54.76.pth.


[Train Epoch 2]: 100%|██████████| 399/399 [10:29<00:00,  1.58s/it]


Epoch 2: Train Loss: 453.3245 | Train Acc: 58.29%
Epoch 2: Validation Accuracy: 63.77%

Model saved to ./files/vit_model_weight_+1epoch_63.77.pth.


...

[Train Epoch 12]: 100%|██████████| 399/399 [10:23<00:00,  1.56s/it]


Epoch 12: Train Loss: 45.5698 | Train Acc: 96.39%
Epoch 12: Validation Accuracy: 90.73%

Model saved to ./files/vit_model_weight_+11epoch_90.73.pth.


[Train Epoch 13]: 100%|██████████| 399/399 [10:25<00:00,  1.57s/it]


Epoch 13: Train Loss: 25.4112 | Train Acc: 98.08%
Epoch 13: Validation Accuracy: 91.82%

Model saved to ./files/vit_model_weight_+12epoch_91.82.pth.


[Train Epoch 14]:  43%|████▎     | 170/399 [04:29<05:54,  1.55s/it]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [13]:
model_save_dir = './files'
os.makedirs(model_save_dir, exist_ok=True)
model_save_path = os.path.join(model_save_dir, 'vit_model_weight.pth')
torch.save(model.state_dict(), model_save_path)
print(f'Model saved to {model_save_path}.')


Model saved to ./files/vit_model_weight.pth.


In [10]:
# model = ImageSetViT() 
# + 추가 학습
trained_model = train_model(model, train_loader, val_loader, device, num_epochs=50, lr=1e-4)


[Train Epoch 1]: 100%|██████████| 290/290 [08:48<00:00,  1.82s/it]


Epoch 1: Train Loss: 135.3117 | Train Acc: 79.83%
Epoch 1: Validation Accuracy: 73.96%



[Train Epoch 2]: 100%|██████████| 290/290 [08:32<00:00,  1.77s/it]


Epoch 2: Train Loss: 118.3377 | Train Acc: 81.62%
Epoch 2: Validation Accuracy: 79.84%



[Train Epoch 3]: 100%|██████████| 290/290 [08:37<00:00,  1.79s/it]


...

Epoch 48: Train Loss: 69.7971 | Train Acc: 89.89%
Epoch 48: Validation Accuracy: 82.49%



[Train Epoch 49]: 100%|██████████| 290/290 [10:09<00:00,  2.10s/it]


Epoch 49: Train Loss: 71.8248 | Train Acc: 89.11%
Epoch 49: Validation Accuracy: 88.48%



[Train Epoch 50]: 100%|██████████| 290/290 [09:17<00:00,  1.92s/it]


Epoch 50: Train Loss: 77.8933 | Train Acc: 88.62%
Epoch 50: Validation Accuracy: 87.44%



In [12]:
model_save_dir = './files'
os.makedirs(model_save_dir, exist_ok=True)
model_save_path = os.path.join(model_save_dir, 'vit_custom_model_weight_eth_100_2.pth')
torch.save(model.state_dict(), model_save_path)
print(f'Model saved to {model_save_path}.')


Model saved to ./files/vit_custom_model_weight_eth_100_2.pth.


In [ ]:
# model = ImageSetViT() 
# + 추가 학습
trained_model = train_model(model, train_loader, val_loader, device, num_epochs=100, lr=1e-4)


In [ ]:
model = ImageSetViT()  
trained_model = train_model(model, train_loader, val_loader, device, num_epochs=30, lr=1e-3)


In [ ]:
# 2. object detection 패치 단위로 안하고, 키트만 detection 해서, ViT 모델에 키트 데이터 넣을 경우 어떻게 결과가 나올지?

# dataset 정의

# ViT 모델 로드



In [ ]:
# 3. classical detection (opencv, 색상 기반)

# object detection으로 패치에 대해서 detection하고, color 기반으로 휴리스틱하게 판정을 내리기?

In [32]:
## ViT 모델 학습시 코드 뼈대 작성
## 학습 :
## object detection 모델에 사용되는 label이 X data를 구성하는 데에 사용됨.
## dir 이름이 Y data를 구성하는 데에 사용됨.

## 검증 :
## 학습과 동일

## 참고) 테스트 :
## object detection 모델의 예측 결과가 X data를 구성하는 데에 직접적으로 사용됨
## Y data는 모델의 예측 결과

import glob
from PIL import Image
import numpy as np
import torch
import torchvision.transforms as T

data_path = "/home/jupyter-heypaprika/datasets/urin_check_dataset/Z-FLIP_unlabelled/"
files = glob.glob(data_path+"*.txt")
image_file_size = glob.glob(data_path+"*.jpg")[0]
image = Image.open(image_file_size).convert("RGB")
np_image = np.array(image)

img_w, img_h = image.size

### X : image to tensor ###

with open(files[1], "r") as file:
    cur_label_xywhn = list(map(str.rstrip, file.readlines()))

print(cur_label_xywhn)

# to transform xywhn to xyxy
def xywhn2xyxy(x, y, w, h, img_w, img_h):
    x1 = (x - w / 2) * img_w
    y1 = (y - h / 2) * img_h
    x2 = (x + w / 2) * img_w
    y2 = (y + h / 2) * img_h
    return x1, y1, x2, y2

cur_label = ["" for _ in range(len(cur_label_xywhn))]

for i in range(len(cur_label)):
    n, x, y, w, h = map(float, cur_label_xywhn[i].split())
    x1, y1, x2, y2 = xywhn2xyxy(x, y, w, h, img_w, img_h)
    cur_label[i] = [x1, y1, x2, y2, int(n)]


xyxy = np.array(cur_label)
sorted_data = xyxy[np.lexsort((xyxy[:,1], xyxy[:,0]))]


kit = sorted_data[:3]
sensor = sorted_data[3:]
kit = kit[np.lexsort((kit[:,0], kit[:,1]))]
groups = []

for k in kit:
    # k[0]: lower bound for x, k[2]: upper bound for x
    # k[1]: lower bound for y, k[3]: upper bound for y
    mask = (sensor[:, 0] > k[0]) & (sensor[:, 0] < k[2]) & \
           (sensor[:, 1] > k[1]) & (sensor[:, 1] < k[3])
    result = sensor[mask]
    result = result[np.lexsort((result[:,1], result[:,0]))]

    groups.append(result)


image_size = 32

transform = T.Compose([
            T.Resize((image_size, image_size)),
            T.ToTensor(),  # Converts to (C, H, W)
        ])


# 각 그룹 출력
batch = []
for group in groups:
    print(f"cur Kit 에 해당하는 sensor 데이터:")
    images = []
    for j, patch in enumerate(group):
        print(patch)
        x1, y1, x2, y2, n = patch
        crop_patch = np_image[round(y1):round(y2), round(x1):round(x2)]
        crop_patch = crop_patch[..., ::-1]
        # v1: 전체 이미지를 그대로 리사이징
        crop_patch_pil = Image.fromarray(crop_patch)
        img = transform(crop_patch_pil)  # (3, 32, 32)
        images.append(img)
    images = torch.stack(images)
    batch.append(images)
batch = torch.stack(batch)

print(batch.shape)

### X : image to tensor, Done ###



### Y : dir to label

['0 0.550258 0.482883 0.792669 0.077674', '1 0.617799 0.486476 0.031260 0.037913', '1 0.329770 0.677449 0.032549 0.039392', '1 0.333755 0.299361 0.033366 0.040691', '0 0.549963 0.680188 0.791086 0.061061', '1 0.525079 0.300167 0.033444 0.041095', '1 0.187318 0.674683 0.032913 0.038827', '1 0.472916 0.680706 0.032978 0.040174', '1 0.572871 0.300507 0.033263 0.040777', '1 0.188825 0.470396 0.032049 0.037992', '1 0.381804 0.299490 0.033687 0.041507', '1 0.520465 0.681194 0.032542 0.039607', '1 0.235907 0.471770 0.032014 0.037578', '1 0.237901 0.298994 0.033299 0.040819', '1 0.615836 0.683158 0.032793 0.040101', '1 0.477110 0.299988 0.033037 0.040756', '1 0.620210 0.300888 0.032191 0.040185', '1 0.331899 0.475316 0.030721 0.036569', '1 0.190821 0.299361 0.033619 0.040581', '1 0.475174 0.480683 0.030734 0.036728', '1 0.283940 0.473679 0.031935 0.037647', '1 0.281818 0.676706 0.032514 0.039516', '0 0.552543 0.300242 0.789979 0.051366', '1 0.567939 0.681674 0.032169 0.039883', '1 0.429138 0.2